# Kinstretch Pose Analysis

Pipeline for analyzing kinstretch movement videos:
1. Search YouTube for kinstretch videos
2. Download a video
3. Extract pose landmarks using MediaPipe
4. Visualize and analyze movement patterns

In [ ]:
# Install dependencies (run once in Colab)
# !pip install yt-dlp youtube-transcript-api mediapipe opencv-python pandas matplotlib pydantic

In [ ]:
import pandas as pd

from kinstretch.youtube import search_videos, download_video
from kinstretch.pose_extraction import extract_poses, download_model
from kinstretch.visualization import plot_pose, animate_poses, plot_joint_progression

## 1. Search YouTube for Kinstretch Videos

In [ ]:
videos = search_videos("kinstretch", max_results=100)

df = pd.DataFrame([v.model_dump() for v in videos])
print(f"Found {len(df)} videos")
print(f"Unique creators: {df['creator'].nunique()}")
df.head()

## 2. Download a Video

In [ ]:
# Pick a video to analyze
video_url = df.iloc[1]["url"]
print(f"Downloading: {df.iloc[1]['title']}")

video_path = download_video(video_url)
print(f"Saved to: {video_path}")

## 3. Extract Pose Landmarks

In [ ]:
# Download model if needed, then extract poses
model_path = download_model()
poses = extract_poses(video_path, model_path=model_path, frame_stride=5)

print(f"Extracted {len(poses)} pose frames")
print(f"Duration: {poses[-1].timestamp_ms / 1000:.1f}s")

## 4. Visualize a Single Pose

In [ ]:
plot_pose(poses[0].landmarks, title="Kinstretch Frame 1")

## 5. Animate Pose Sequence

In [ ]:
# Keep a reference to prevent garbage collection
anim = animate_poses(poses, interval=200)

In [ ]:
# Optionally save as GIF
# anim = animate_poses(poses, save_path="kinstretch_poses.gif", fps=5)

## 6. Joint Progression Over Time

In [ ]:
# Shoulders and hips by default
plot_joint_progression(poses)

# Or specify custom joints (e.g. knees and ankles)
# plot_joint_progression(poses, joint_indices=[25, 26, 27, 28])

## 7. Summary Statistics

In [ ]:
from kinstretch.models import VideoMetadata, VideoAnalysis

analysis = VideoAnalysis(
    metadata=videos[1],
    poses=poses,
)

print(f"Title:    {analysis.metadata.title}")
print(f"Creator:  {analysis.metadata.creator}")
print(f"Frames:   {analysis.n_frames}")
print(f"Duration: {analysis.duration_seconds:.1f}s")